In [1]:
import pandas as pd

In [3]:
businesses = pd.read_csv("business.csv") 

In [4]:
columns = ["attributes","business_id","categories","city","is_open","latitude","longitude","name","review_count","stars"]
businesses = businesses.loc[:,columns]

In [5]:
businesses_nlv = businesses.loc[(businesses["city"]=="North Las Vegas") & (businesses["is_open"]==1) ,:]

In [6]:
businesses_nlv = businesses_nlv.drop("is_open",axis = 1)

In [7]:
businesses_nlv = businesses_nlv.reset_index(drop=True)

In [8]:
businesses_nlv.to_csv("businesses_nlv.csv")

In [16]:
tmp = businesses_nlv.loc[:, ["business_id"]]

In [23]:
tmp.to_csv("../business_tmp.csv", index=False, header=False)

In [2]:
reviews = pd.read_csv("review.csv")

In [3]:
reviews['business_id'] = reviews['business_id'].str[1:]

In [4]:
reviews.to_csv("review_nlv.csv", index=False)

In [5]:
b = pd.read_csv("businesses_nlv.csv") 
r = pd.read_csv("review_nlv.csv") 

,text,date,cool,funny,stars,business_id,useful,user_id,review_id
0,Had the Tonkotsu ramen for lunch upon the reco...,2017-05-02 20:37:33 UTC,0,0,4,DnaKAs2oK3rXfrjSvn9ew,0,70p7ZC7JNkAn4YHJL2oLqQ,SvrM4R0Sn1085JldYArCwA
1,"Service was quick, food was excellent am glad ...",2015-10-09 01:30:39 UTC,0,0,5,DnaKAs2oK3rXfrjSvn9ew,0,vRhxHrWwHpPDbzzKsCKIQw,6p-MEBAuZ4dhOA1k24vgdw
2,"I actually think that the place is a 3.5, so, ...",2015-06-07 00:33:50 UTC,0,0,4,DnaKAs2oK3rXfrjSvn9ew,0,Bxf1jhd66YvcI0GFTIh3yQ,5P--fn1aEuuzsRiIgRcfCA
3,I eat alot of ramen and alot of curry.. but i ...,2017-07-22 20:35:05 UTC,0,0,5,DnaKAs2oK3rXfrjSvn9ew,0,eRwwZ7CjgDuywdNZ_mtgKQ,XJLHswUbJYbzj9CmHpIsSw
4,Stopped in on a whim with my husband. Our stom...,2016-01-03 02:22:19 UTC,0,0,5,DnaKAs2oK3rXfrjSvn9ew,0,PBSkpQxdUzx-4LyvEgX1KQ,FieX2hOMO2R0z5hj59sXCg


In [9]:
b.review_count.isnull().any()

False

In [11]:
b.review_count.sum()

43826

In [12]:
len(r)

45606